## Building Q&A application using Knowledge Bases for Amazon Bedrock - RetrieveAndGenerate API
### Context

With knowledge bases, you can securely connect foundation models (FMs) in Amazon Bedrock to your company
data for Retrieval Augmented Generation (RAG). Access to additional data helps the model generate more relevant,
context-speciﬁc, and accurate responses without continuously retraining the FM. All information retrieved from
knowledge bases comes with source attribution to improve transparency and minimize hallucinations. For more information on creating a knowledge base using console, please refer to this [post](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html).

In this notebook, we will dive deep into building Q&A application using `RetrieveAndGenerate` API provided by Knowledge Bases for Amazon Bedrock. This API will query the knowledge base to get the desired number of document chunks based on similarity search, integrate it with Large Language Model (LLM) for answering questions.


### Pattern

We can implement the solution using Retreival Augmented Generation (RAG) pattern. RAG retrieves data from outside the language model (non-parametric) and augments the prompts by adding the relevant retrieved data in context. Here, we are performing RAG effectively on the knowledge base created in the previous notebook or using console. 

### Pre-requisite

Before being able to answer the questions, the documents must be processed and stored in knowledge base.

1. Load the documents into the knowledge base by connecting your s3 bucket (data source). 
2. Ingestion - Knowledge base will split them into smaller chunks (based on the strategy selected), generate embeddings and store it in the associated vectore store and notebook [0_create_ingest_documents_test_kb.ipynb](./0\_create_ingest_documents_test_kb.ipynb) takes care of it for you.

![data_ingestion.png](./images/data_ingestion.png)


#### Notebook Walkthrough

For our notebook we will use the `RetreiveAndGenerate API` provided by Knowledge Bases for Amazon Bedrock which converts user queries into
embeddings, searches the knowledge base, get the relevant results, augment the prompt and then invoking a LLM to generate the response. 

We will use the following workflow for this notebook. 

![retrieveAndGenerate.png](./images/retrieveAndGenerate.png)


### USE CASE:

#### Dataset

In this example, you will use several years of Amazon's Letter to Shareholders as a text corpus to perform Q&A on. This data is already ingested into the knowledge base. You will need the `knowledge base id` and `model ARN` to run this example. We are using `Anthropic Claude 3 Haiku` model for generating responses to user questions.

### Python 3.10

⚠  For this lab we need to run the notebook based on a Python 3.10 runtime. ⚠

### Setup

Install following packages. 

In [1]:
%pip install --upgrade pip
%pip install boto3==1.33.2 --force-reinstall --quiet
%pip install botocore==1.33.2 --force-reinstall --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.13.1 requires botocore<1.34.132,>=1.34.70, but you have botocore 1.33.13 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.1 which is incompatible.
autogluon-common 0.8.3 requires pandas<1.6,>=1.4.1, but you have pandas 2.1.4 which is incompatible.
autogluon-core 0.8.3 requires pandas<1.6,>=1.4.1, but you have pandas 2.1.4 which is incompatible.
autogluon-core 0.8.3 requires scikit-learn<1.4.1,>=1.1, but you have scikit-learn 1.4.2 which is incompatible.

In [2]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
%store -r

In [10]:
import boto3
import pprint
from botocore.client import Config

pp = pprint.PrettyPrinter(indent=2)

bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config)
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name

model_id = "anthropic.claude-3-haiku-20240307-v1:0" # try with both claude 3 Haiku as well as claude 3 Sonnet. for claude 3 Sonnet - "anthropic.claude-3-sonnet-20240229-v1:0"
region_id = region_name # replace it with the region you're running sagemaker notebook

## RetreiveAndGenerate API
Behind the scenes, `RetrieveAndGenerate` API converts queries into embeddings, searches the knowledge base, and then augments the foundation model prompt with the search results as context information and returns the FM-generated response to the question. For multi-turn conversations, Knowledge Bases manage short-term memory of the conversation to provide more contextual results. 

The output of the `RetrieveAndGenerate` API includes the   `generated response`, `source attribution` as well as the `retrieved text chunks`. 

In [11]:
def retrieveAndGenerate(input, kbId, sessionId=None, model_id = "anthropic.claude-3-haiku-20240307-v1:0", region_id = "us-east-1"):
    model_arn = f'arn:aws:bedrock:{region_id}::foundation-model/{model_id}'
    if sessionId:
        return bedrock_agent_client.retrieve_and_generate(
            input={
                'text': input
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kbId,
                    'modelArn': model_arn
                }
            },
            sessionId=sessionId
        )
    else:
        return bedrock_agent_client.retrieve_and_generate(
            input={
                'text': input
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kbId,
                    'modelArn': model_arn
                }
            }
        )

In [12]:
query = "วงประชุม PAM จัดขึ้นล่าสุดตอนไหน?"
response = retrieveAndGenerate(query, kb_id,model_id=model_id,region_id=region_id)
generated_text = response['output']['text']
pp.pprint(generated_text)

('ตามระเบียบวาระการประชุมคณะกรรมการบริหาร สปสช. ครั้งที่ 9/2566 วันที่ 27 '
 'มิถุนายน 2566, ระเบียบวาระการประชุมผู้บริหาร สปสช. (Policy Advocacy Meeting: '
 'PAM) ครั้งที่ 2/2566 จัดขึ้นในวันจันทร์ที่ 10 กรกฎาคม 2566.')


In [13]:
citations = response["citations"]
contexts = []
for citation in citations:
    retrievedReferences = citation["retrievedReferences"]
    for reference in retrievedReferences:
         contexts.append(reference["content"]["text"])

pp.pprint(contexts)

[ 'ระเบียบวาระการประชุมคณะกรรมการบรหิาร สปสช. ครั้งท่ี ๙/๒๕๖๖ วันท่ี ๒๗ '
  'มิถุนายน ๒๕๖๖                                   ๑/๒             '
  'ระเบียบวาระประชุมผู\uf70bบริหาร สปสช. (Policy Advocacy Meeting: PAM) '
  'คร้ังที่ ๒/๒๕๖๖  วันจันทร\uf70eที่ ๑๐ กรกฎาคม ๒๕๖๖ เวลา ๑๓.๐๐ – ๑๖.๐๐ น.    '
  'ณ ห\uf70bองประชุม ๒๐๒ ชั้น ๒ สำนักงานหลักประกันสขุภาพแห\uf70aงชาติ '
  'และผ\uf70aานสื่ออิเล็กทรอนิกส\uf70e      o ประธานที่ประชุม : นพ. จเด็จ '
  'ธรรมธัชอารี  เลขาธิการ สปสช.']


In [14]:
citations

[{'generatedResponsePart': {'textResponsePart': {'text': 'ตามระเบียบวาระการประชุมคณะกรรมการบริหาร สปสช. ครั้งที่ 9/2566 วันที่ 27 มิถุนายน 2566, ระเบียบวาระการประชุมผู้บริหาร สปสช. (Policy Advocacy Meeting: PAM) ครั้งที่ 2/2566 จัดขึ้นในวันจันทร์ที่ 10 กรกฎาคม 2566.',
    'span': {'start': 0, 'end': 207}}},
  'retrievedReferences': [{'content': {'text': 'ระเบียบวาระการประชุมคณะกรรมการบรหิาร สปสช. ครั้งท่ี ๙/๒๕๖๖ วันท่ี ๒๗ มิถุนายน ๒๕๖๖                                   ๑/๒             ระเบียบวาระประชุมผู\uf70bบริหาร สปสช. (Policy Advocacy Meeting: PAM) คร้ังที่ ๒/๒๕๖๖  วันจันทร\uf70eที่ ๑๐ กรกฎาคม ๒๕๖๖ เวลา ๑๓.๐๐ – ๑๖.๐๐ น.    ณ ห\uf70bองประชุม ๒๐๒ ชั้น ๒ สำนักงานหลักประกันสขุภาพแห\uf70aงชาติ และผ\uf70aานสื่ออิเล็กทรอนิกส\uf70e      o ประธานที่ประชุม : นพ. จเด็จ ธรรมธัชอารี  เลขาธิการ สปสช.'},
    'location': {'type': 'S3',
     's3Location': {'uri': 's3://bedrock-kb-us-west-2-615631871157/0_วาระการประชุม PAM 2_66.pdf'}}}]}]

## Next Steps

If you want more customized experience, you can use `Retrieve API`. This API converts user queries into embeddings, searches the knowledge base, and returns the relevant results, giving you more control to build custom workflows on top of the semantic search results. 
For sample code, try following notebooks: 
- [2_Langchain-rag-retrieve-api-mistral-and-claude-3-haiku.ipynb](./2_Langchain-rag-retrieve-api-mistral-and-claude-3-haiku.ipynb) - it calls the `retrieve` API to get relevant contexts and then augment the context to the prompt, which you can provide as input to any text-text model provided by Amazon Bedrock. 
  
- You can use the RetrieveQA chain from LangChain and add Knowledge Base as retriever. For sample code, try notebook: [3_Langchain-rag-retrieve-api-claude-3.ipynb](./3_Langchain-rag-retrieve-api-claude-3.ipynb)



<div class="alert alert-block alert-warning">
<b>Next steps:</b> Proceed to the next labs to learn how to use Bedrock Knowledge bases with Langchain and Claude. Remember to CLEAN_UP at the end of your session.
</div>